In [21]:
import pandas as pd
import numpy as np 
from tqdm import tqdm
from scipy.sparse import csr_matrix, lil_matrix, load_npz

In [22]:
path = '../../data/Mellody_tuner/images/output_images/pseudolabels/matrices/cls/cls_T11_fold_vector.npy'
folds = np.load(path)

In [23]:
path = '../../data/Mellody_tuner/images/output_images/pseudolabels/matrices/cls/cls_T10_y.npz'
t10 = load_npz(path).tocsr()

In [24]:
out = lil_matrix(t10) 
out

<10077x285 sparse matrix of type '<class 'numpy.float32'>'
	with 184789 stored elements in List of Lists format>

In [25]:
# fold 2

In [26]:
fold = 2 
out[folds != fold, :] = 0 

In [27]:
out

<10077x285 sparse matrix of type '<class 'numpy.float32'>'
	with 36671 stored elements in List of Lists format>

In [28]:
# remove auxiliary tasks (aggregation weight 0)

In [29]:
path = '../../data/Mellody_tuner/images/output_images/pseudolabels/results_tmp/classification/T8c.csv'
t8c = pd.read_csv(path)

In [30]:
t8c.columns

Index(['cont_classification_task_id', 'classification_task_id',
       'input_assay_id', 'assay_type', 'variance_quorum_OK', 'is_auxiliary',
       'use_in_regression', 'threshold', 'threshold_method', 'direction',
       'training_quorum_OK', 'evaluation_quorum_OK', 'aggregation_weight',
       'weight', 'num_total_actives', 'num_fold_min_actives',
       'num_total_inactives', 'num_fold_min_inactives', 'n_tasks'],
      dtype='object')

In [31]:
cctis = np.array(t8c.query('is_auxiliary == False').query('aggregation_weight > 0')['cont_classification_task_id'],dtype=int)


In [32]:
cctis.shape

(89,)

In [33]:
np.save('./files/main_tasks_with_image.npy',cctis)

In [34]:
out.shape

(10077, 285)

In [35]:
# mind the ~
mask = ~np.isin(
    np.array(range(int(t8c['cont_classification_task_id'].max()+1)))
    ,cctis
)

In [36]:
out[:,mask].shape

(10077, 196)

In [37]:
# ignore all non-main tasks 
out[:,mask] = 0 

In [38]:
out

<10077x285 sparse matrix of type '<class 'numpy.float32'>'
	with 15938 stored elements in List of Lists format>

In [39]:
filename = './files/y_sparse_step1_main_tasks_fold2.npy'
np.save(filename,csr_matrix(out))

In [40]:
np.load(filename, allow_pickle=True).item().tocsr()

<10077x285 sparse matrix of type '<class 'numpy.float32'>'
	with 15938 stored elements in Compressed Sparse Row format>